In [6]:
import nltk

In [ ]:
import gensim
w2v = gensim.models.KeyedVectors.load_word2vec_format("/Users/janfan/Downloads/GoogleNews-vectors-negative300-SLIM.bin.gz",
                                                      binary=True)

In [17]:
class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

    
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.vectors[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

In [ ]:
import pandas as pd, numpy as np
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
subm = pd.read_csv('data/sample_submission.csv')

In [18]:
mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2v)

In [19]:
train_mean_embedded = mean_embedding_vectorizer.fit_transform(train['comment_text'])

In [23]:
test_mean_embedded = mean_embedding_vectorizer.transform(test['comment_text'])

In [25]:
from sklearn.linear_model import LogisticRegression

label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((len(test), len(label_cols)))
for i, j in enumerate(label_cols):
    m = LogisticRegression()
    mf = m.fit(train_mean_embedded, train[j])
    preds[:,i] = mf.predict_proba(test_mean_embedded)[:,1]

In [26]:
submid = pd.DataFrame({'id': subm["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
submission.to_csv('submission/submission-w2v-docmean-lr.csv', index=False)